In [1]:
from evaluation.evaluate import (
    get_froc_vals,
    convert_mm_to_pixel,
    get_F1_scores,
)
from monkey.data.data_utils import open_json_file, extract_id
import os
from pprint import pprint

/home/u1910100/miniconda3/envs/tiatoolbox/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/u1910100/miniconda3/envs/tiatoolbox/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
|2024-11-04|15:55:44.402| [WARNING] /home/u1910100/miniconda3/envs/tiatoolbox/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-f

In [5]:
GROUND_TRUTH_DIRECTORY = (
    "/home/u1910100/Downloads/Monkey/annotations/json"
)
FOLD = 3
model_name = "efficientunetb0"
PREDICT_DIR = f"/home/u1910100/Documents/Monkey/local_output/{model_name}/Fold_{FOLD}"
SPACING_LEVEL0 = 0.24199951445730394

split_info = open_json_file(
    "/home/u1910100/Documents/Monkey/patches_256/wsi_level_split.json"
)

val_wsi_files = split_info[f"Fold_{FOLD}"]["test_files"]

print(val_wsi_files)

sum_score = 0.0
sum_f1 = 0.0

for wsi_name in val_wsi_files:
    wsi_name_without_ext = os.path.splitext(wsi_name)[0]
    wsi_id = extract_id(wsi_name)

    gt_inf_cells = open_json_file(
        json_path=os.path.join(
            GROUND_TRUTH_DIRECTORY,
            f"{wsi_id}_inflammatory-cells.json",
        )
    )

    location_detected_inflammatory_cells = os.path.join(
        PREDICT_DIR, f"{wsi_id}_detected-inflammatory-cells.json"
    )

    result_detected_inflammatory_cells = open_json_file(
        json_path=location_detected_inflammatory_cells,
    )
    result_detected_inflammatory_cells = convert_mm_to_pixel(
        result_detected_inflammatory_cells
    )

    inflamm_froc = get_froc_vals(
        gt_inf_cells,
        result_detected_inflammatory_cells,
        radius=int(7.5 / SPACING_LEVEL0),
    )

    inflamm_f1 = get_F1_scores(
        gt_inf_cells,
        result_detected_inflammatory_cells,
        radius=int(7.5 / SPACING_LEVEL0),
    )

    # pprint(wsi_id)
    # pprint(f"FROC score: {inflamm_froc['froc_score_slide']}")
    sum_score += inflamm_froc["froc_score_slide"]
    sum_f1 += inflamm_f1["F1"]

pprint(f"Average FROC = {sum_score / len(val_wsi_files)}")
pprint(f"Average F1 = {sum_f1 / len(val_wsi_files)}")

['A_P000022_PAS_CPG.tif', 'A_P000024_PAS_CPG.tif', 'A_P000029_PAS_CPG.tif', 'A_P000030_PAS_CPG.tif', 'A_P000031_PAS_CPG.tif', 'A_P000032_PAS_CPG.tif', 'B_P000011_PAS_CPG.tif', 'B_P000012_PAS_CPG.tif', 'B_P000013_PAS_CPG.tif', 'B_P000014_PAS_CPG.tif', 'B_P000015_PAS_CPG.tif', 'C_P000031_PAS_CPG.tif', 'C_P000032_PAS_CPG.tif', 'C_P000033_PAS_CPG.tif', 'C_P000034_PAS_CPG.tif', 'C_P000035_PAS_CPG.tif', 'D_P000011_PAS_CPG.tif', 'D_P000012_PAS_CPG.tif', 'D_P000013_PAS_CPG.tif', 'D_P000014_PAS_CPG.tif']
'Average FROC = 0.22963233908920685'
'Average F1 = 0.6289410948462979'


In [29]:
file_id = "A_P000001"


# gt_lymphocytes = open_json_file(
#     json_path=os.path.join(
#         GROUND_TRUTH_DIRECTORY, f"{file_id}_lymphocytes.json"
#     )
# )
# gt_monocytes = open_json_file(
#     json_path=os.path.join(
#         GROUND_TRUTH_DIRECTORY, f"{file_id}_monocytes.json"
#     )
# )
gt_inf_cells = open_json_file(
    json_path=os.path.join(
        GROUND_TRUTH_DIRECTORY, f"{file_id}_inflammatory-cells.json"
    )
)


# location_detected_lymphocytes =
# location_detected_monocytes =
location_detected_inflammatory_cells = "/home/u1910100/Documents/Monkey/local_output/detected-inflammatory-cells.json"

# result_detected_lymphocytes = open_json_file(
#         location=location_detected_lymphocytes,
#     )
# result_detected_lymphocytes = convert_mm_to_pixel(
# result_detected_lymphocytes
# )

# result_detected_monocytes = open_json_file(
# location=location_detected_monocytes,
# )
# result_detected_monocytes = convert_mm_to_pixel(
# result_detected_monocytes
# )

result_detected_inflammatory_cells = open_json_file(
    json_path=location_detected_inflammatory_cells,
)
result_detected_inflammatory_cells = convert_mm_to_pixel(
    result_detected_inflammatory_cells
)

# # compare the results to your ground truth and compute some metrics
# lymphocytes_froc = get_froc_vals(
#     gt_lymphocytes,
#     result_detected_lymphocytes,
#     radius=int(4 / SPACING_LEVEL0),
# )  # margin for lymphocytes is 4um at spacing 0.25 um / pixel
# monocytes_froc = get_froc_vals(
#     gt_monocytes, result_detected_monocytes, radius=int(10 / 0.25)
# )  # margin for monocytes is 10um at spacing 0.25 um / pixel
inflamm_froc = get_froc_vals(
    gt_inf_cells,
    result_detected_inflammatory_cells,
    radius=int(7.5 / SPACING_LEVEL0),
)  # margin for inflammatory cells is 7.5um at spacing 0.24 um / pixel

pprint(f"FROC score: {inflamm_froc['froc_score_slide']}")
# pprint(f"sensitivity_slide: {inflamm_froc['sensitivity_slide']}")
# pprint(f"fp_per_mm2_slide: {inflamm_froc['fp_per_mm2_slide']}")

'FROC score: 0.20829616413916144'
